In [ ]:
# 1. CLEANUP & INSTALL TOOLS
print("🧹 [1/3] Cleaning system & installing web tools...")
!rm -rf /etc/apt/sources.list.d/r2u.list
!apt-get update -qq > /dev/null
!apt-get install -y adb wget nodejs npm -qq > /dev/null

# 2. START THE WEB VIEWER DIRECTLY (NO DOCKER)
print("🌐 [2/3] Starting Web Interface...")
import os, subprocess, time, threading
# We use npx to run the viewer as a standard linux process
os.system('npx -y ws-scrcpy-docker --port 7860 > viewer.log 2>&1 &')
time.sleep(10)

# 3. GENERATE CLOUDFLARE LINK
print("🔗 [3/3] Generating Tunnel Link...")
!wget -q -O cf https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cf

print("\n" + "="*60)
print("🚀 SEARCH FOR THE LINK ENDING IN .trycloudflare.com BELOW")
print("="*60 + "\n")

def start_tunnel():
    os.system('./cf tunnel --url http://localhost:7860')

threading.Thread(target=start_tunnel, daemon=True).start()
time.sleep(10)

# Force print the link so it's not buried
!grep -o 'https://[-a-zA-Z0-9.]*\.trycloudflare\.com' <(timeout 5s ./cf tunnel --url http://localhost:7860 2>&1)